Provide Valid Facebook Cookies:
If you need to scrape non-public profiles, you'll need to provide valid Facebook cookies. Here's how you can do that:

a. Log in to your Facebook account in a web browser.

b. After logging in, open the browser's developer tools (usually by pressing F12 or right-clicking and selecting "Inspect").

c. Go to the "Network" or "Console" tab, and find the request to www.facebook.com.

d. In the request headers, look for the cookie field, and copy the entire cookie value.

e. Pass the cookie value to the facebook_scraper function using the cookies parameter:


In [8]:
# from facebook_scraper import get_profile
# cookies = "cookies.txt"
# my_profile = get_profile("yitong.tseo", cookies=cookies, friends=True)

# import os
# import requests
# from io import BytesIO
# from PIL import Image

# num_errors = 0
# for node in my_profile['Friends']:
#     # Extract the relevant information
#     name = node['name']
#     profile_pic_url = node["profile_picture"]
#     img_path = os.path.join('friend_pics', f"{name.replace(' ', '_')}.png")

#     if not os.path.exists(img_path):
#         # Load the profile picture
#         try:
#             response = requests.get(profile_pic_url)
#             img = Image.open(BytesIO(response.content))
#             # Scale the image to fit the node
#             print(f"Saving {name}'s picture!")
            
#             img.save(img_path)
#         except:
#             num_errors += 1
#     else:
#         img = Image.open(img_path)




Saving James Ko's picture!
Saving Danial Hafiz's picture!
Saving Pantea Karimi's picture!
Saving Nathan Thompson's picture!
Saving Joana Genova-Rudiakov's picture!
Saving Brittany Gelb's picture!
Saving Jonathan Engel's picture!
Saving Samantha Godoy's picture!
Saving Sorcha McCarrey's picture!
Saving Jenny Dew's picture!
Saving Ariel Chu's picture!
Saving Bijan Mazaheri's picture!
Saving Harrison Kerch's picture!
Saving Parmida Davarmanesh's picture!
Saving Fadi Botros's picture!
Saving Taryn Johnson's picture!
Saving Nathaniel Archie's picture!
Saving Shannon Kruczek's picture!
Saving Jamal Meneide's picture!
Saving Dana Williams's picture!
Saving Akshay Subramanian's picture!
Saving Brittany Bryant's picture!
Saving Preston Smith's picture!
Saving Kat Lansang's picture!
Saving Gabriella Gonzalez's picture!
Saving Janet Walker's picture!
Saving Alex LeNail's picture!
Saving Chris Janson's picture!
Saving Intekhab Hossain's picture!
Saving Caroline Atwood's picture!
Saving Michelle Li

In [ ]:
my_profile.keys()

In [17]:
from pyvis.network import Network
import networkx as nx
import pickle

# Load your graph
with open('state.pickle', 'rb') as file:
    state = pickle.load(file)
    friends_graph = state['graph']

your_node = 'Yitong'

if your_node in friends_graph:
    friends_graph.remove_node(your_node)
    print(f"Node '{your_node}' removed successfully.")
else:
    print(f"Node '{your_node}' not found in the graph.")



# Create a Pyvis network
nt = Network(notebook=True, height="750px", width="100%")
nt.from_nx(friends_graph)
nt.show("friends_network.html")


Node 'Yitong' removed successfully.
friends_network.html


# TODO: manually add Hui Fu, Simone Fary, Yilan Tseo, Changyan Wang...

In [1]:
from facebook_scraper import get_profile
import networkx as nx
import pickle
from collections import deque
from tqdm import tqdm
import os
import pdb
import time
import random

MAX_DEPTH = 1
# Sleep from 30 minutes to an hour...
SLEEP_TIME_RANGE = (int(3600 / 3.1), int(3600 / 1.8))
SKIP_LIST = [
    "Robert Hart",
    "Janet Walker",
    "Matthew Quinn",
    "Christian Schmidt",
    "Nico Dols",
]


def get_friends_graph(
    user_id,
    cookies="cookies.txt",
    user_queue=None,
    visited_nodes=None,
    friends_graph=None,
):
    if friends_graph is None:
        friends_graph = nx.Graph()
    if visited_nodes is None:
        visited_nodes = set()
    if user_queue is None:
        user_queue = deque(
            [(user_id, "Yitong", 0)]
        )  # Initialize the queue with the starting user ID and depth 0
    else:
        user_queue = deque(user_queue)  # Convert the loaded queue to a deque

    while user_queue:
        current_user_id, current_user_name, current_depth = user_queue.popleft()
        print("queue length, ", len(user_queue), " graph size ", len(friends_graph))
        print(
            "popped ",
            current_user_name,
            " from queue ... are they new?",
            current_user_id not in visited_nodes,
        )

        if current_user_id not in visited_nodes and current_depth <= MAX_DEPTH:
            visited_nodes.add(current_user_id)
            if current_user_name in SKIP_LIST:
                continue

            user_profile = get_profile(current_user_id, cookies=cookies, friends=True)
            print(f"Visiting {user_profile['Name']}")

            user_data = {
                "user_id": user_profile["id"],
                "name": user_profile["Name"],
                "profile_pic": user_profile.get("profile_picture", None),
            }
            friends_graph.add_node(current_user_name, **user_data)

            for friend in tqdm(user_profile["Friends"]):
                friend_id = str(friend["id"])
                friend_name = friend["name"]
                if friend_id not in visited_nodes:
                    friend_data = {
                        "user_id": friend_id,
                        "name": friend_name,
                        "profile_pic": friend.get("profile_picture", None),
                    }
                    if current_depth == MAX_DEPTH:
                        if friend_name in friends_graph:
                            friends_graph.add_edge(current_user_name, friend_name)
                            print(
                                "we just adding an edge from ",
                                current_user_name,
                                " to ",
                                friend_name,
                            )
                        else:
                            print("we are skippin ", friend_name)
                    elif current_depth < MAX_DEPTH:
                        print("we are adding new friend ", friend_name)
                        friends_graph.add_node(friend_name, **friend_data)
                        friends_graph.add_edge(current_user_name, friend_name)
                        user_queue.append((friend_id, friend_name, current_depth + 1))

            # Save the current state
            with open(f"state.pickle", "wb") as file:
                state = {
                    "graph": friends_graph,
                    "user_queue": list(user_queue),
                    "visited_nodes": visited_nodes,
                }
                pickle.dump(state, file)

            time.sleep(random.randint(SLEEP_TIME_RANGE[0], SLEEP_TIME_RANGE[1]))

    return friends_graph


# Check if a saved state exists
if os.path.exists("state.pickle"):
    with open(f"state.pickle", "rb") as file:
        state = pickle.load(file)
        friends_graph = state["graph"]
        user_queue = state["user_queue"]
        visited_nodes = state["visited_nodes"]
        my_friends_graph = get_friends_graph(
            "1685166343",
            user_queue=user_queue,
            visited_nodes=visited_nodes,
            friends_graph=friends_graph,
        )

else:
    # Replace 'your_user_id' with your actual Facebook user ID
    my_friends_graph = get_friends_graph("1685166343")

/opt/anaconda3/envs/tonsil_city/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


queue length,  1187  graph size  1333
popped  Khan Shairani  from queue ... are they new? True


/opt/anaconda3/envs/tonsil_city/lib/python3.9/site-packages/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


Visiting Khan Shairani


100%|██████████| 617/617 [00:00<00:00, 73232.37it/s]

...

queue length,  1180  graph size  1333
popped  Karen Wall Creeden  from queue ... are they new? True
Visiting Karen Wall Creeden


100%|██████████| 617/617 [00:00<00:00, 107564.14it/s]

we are skippin  Christina Park
we are skippin  Anna Li
we are skippin  Grace Lerew
we are skippin  Debra Reilly
we are skippin  Marianna Morris
we are skippin  Stephanie Klotz
we are skippin  Joshua Yang
we are skippin  Benjamin Delin
we are skippin  Rachel Sloan
we are skippin  Andrew Kurtz
we are skippin  Brian Mishler
we are skippin  Dan Yu
we are skippin  Nich Roehler
we are skippin  Parkland TvandFilm
we are skippin  Melissa Lukas
we are skippin  Alex Michael G
we are skippin  Rachael Webb
we are skippin  Jim Tully
we are skippin  Diane Goldstein Stein
we are skippin  Haley Zimring
we are skippin  Jenn Bao
we are skippin  Catherine DeBoeser
we are skippin  Ashleigh Budlong
we are skippin  Scott Watson
we are skippin  Katie LeBoeuf
we are skippin  Alaina Swartz
we are skippin  Inna Eyzerovich
we are skippin  John Fegley
we are skippin  Alice Swearingen Stinebaugh
we are skippin  Tara Lal
we are skippin  Jasmine Lui
we are skippin  Mark Stutz
we are skippin  Diane Wittry
we are skip

queue length,  1179  graph size  1333
popped  Melanie McDougall  from queue ... are they new? True


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [40]:
friends_graph

In [ ]:

# Print the number of nodes (users) and edges (friend connections)
print(f"Number of nodes: {friends_graph.number_of_nodes()}")
print(f"Number of edges: {friends_graph.number_of_edges()}")

# Access the user data for a specific node
user_node = 'Hui Fu'
user_data = friends_graph.nodes[user_node]
print(f"User ID: {user_data['user_id']}")
print(f"Name: {user_data['name']}")
print(f"Profile Picture: {user_data['profile_pic']}")


In [ ]:
# Iterate through all nodes and print their names
for node, data in friends_graph.nodes(data=True):
    print(f"Name: {data['name']}")
